---
# Trabalho 1 - Tópicos Especiais em Matemática Aplicada

**Alunos/Matricula:** João V. Farias & Renan V. Guedes / 221022604 & 221031363

**Arquitetura Usada:** Encoder-Decoder

**Dataset Link:** V1: [D-Talk](https://www.tensorflow.org/datasets/catalog/ted_hrlr_translate#ted_hrlr_translatefr_to_pt) from TensorFlow.Datasets

---

### **Projeto para traduzir mensagens do Francês para o Portugues**  

Neste projeto, vamos explorar e comparar três arquiteturas de redes neurais para tradução automática do francês para o português, usando o *dataset* TED Talks do *Open Translation Project*. A ideia é testar modelos do tipo **Encoder-Decoder**, analisando suas diferenças e impacto na qualidade da tradução.  

Os três modelos que vamos treinar são:  

1. **LSTM (Long Short-Term Memory)**  
   - Um modelo básico de rede recorrente bidirecional. O **Encoder** processa a frase em francês e gera um contexto, enquanto o **Decoder** usa esse contexto para formar a tradução em português.  
   - A principal vantagem desse modelo é sua capacidade de lidar com dependências de longo prazo nas sequências.  

2. **LSTM com Mecanismos de Atenção**  
   - Uma versão aprimorada do modelo anterior, adicionando uma camada de atenção (no nosso caso, escolhemos Luong).
   - A atenção ajuda o modelo a "olhar" para partes específicas da frase de entrada enquanto traduz, melhorando a coerência e precisão.  

3. **Transformers**  
   - Uma abordagem mais moderna, baseada em **autoatenção**, tirando o uso uso de redes recorrentes.  
   - Trabalha com processamento paralelo, usando *Multi-Head Attention* e *Positional Encoding* para entender relações entre palavras, mesmo quando estão distantes na frase.  

**Como vamos testar os modelos?**  
- **Dataset**: Vamos usar cerca de 52.000 pares de frases (francês-português) para treinar, além de 1.200 para validação e 1.800 para teste.  
- **Pré-processamento**: Faremos a tokenização com *SubwordTextEncoder* para reduzir palavras fora do vocabulário (*out-of-vocabulary* – OOV).  
- **Treinamento**: Otimização com Adam, acompanhando o loss e a acurácia durante o processo.  
- **Avaliação**: Vamos comparar os resultados usando a métrica BLEU e analisar exemplos práticos das traduções.  

**O que esperamos encontrar?**  
- entender qual desses modelos tem o melhor equilíbrio entre qualidade de tradução e eficiência computacional.
- provável que o transformer tenha um desempenho superior, já que conseguem processar frases de forma mais eficiente, enquanto o modelo com LSTM e atenção deve mostrar um avanço significativo sobre a versão básica de LSTM.

No fim das contas, tentamos compreender melhor como essas diferentes técnicas vão trazer como resultado.

---

## 📚 Importando as bibliotecas necessárias

In [2]:
# Primeiro, vamos importar todas as bibliotecas que vamos precisar ao longo do Projeto
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os

# Para processamento de texto
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Componentes do Keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Attention, MultiHeadAttention, Concatenate, TimeDistributed
from tensorflow.keras.optimizers import Adam

# Para visualização dos resultados
from sklearn.metrics import confusion_matrix

tf.config.optimizer.set_jit(True)  # Ativa o XLA JIT compilation
tf.keras.mixed_precision.set_global_policy('mixed_float16') # Força o TensorFlow a usar precisão mista

print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.18.0


## 🔍 Carregando e Preparando os Dados

In [3]:
examples, metadata = tfds.load('ted_hrlr_translate/fr_to_pt', with_info=True, as_supervised=True)
train_examples, val_examples = examples['train'], examples['validation']

A gente começou limpando os textos, retirando aqueles símbolos tipo ?, !, . e até o ¿, que no francês e no português não são usados do mesmo jeito. Esses sinais também atrapalham na hora de treinar, então decidimos removê-los pra deixar tudo mais uniforme e fácil de lidar..

In [4]:
def preprocess_text(text):
    text = tf.strings.regex_replace(text, r"([?.!,¿])", r" \1 ")
    text = tf.strings.regex_replace(text, r'[" "]+', " ")
    text = tf.strings.strip(text)
    text = tf.strings.join(['[START]', text, '[END]'], separator=' ')
    return text

A ideia aqui foi organizar os dados de forma que a gente conseguisse trabalhar numa boa com eles. Separamos as frases e garantimos que cada exemplo estivesse no formato certo. Foi interessante ver como uma boa preparação dos dados já facilita bastante o treino dos modelos depois.

In [5]:
def prepare_dataset(examples, max_samples=None):
    fr_texts, pt_texts = [], []
    for fr, pt in examples:
        fr_texts.append(preprocess_text(fr).numpy().decode('utf-8'))
        pt_texts.append(preprocess_text(pt).numpy().decode('utf-8'))
        if max_samples and len(fr_texts) >= max_samples:
            break
    return fr_texts, pt_texts

Pra não sobrecarregar o sistema e também evitar overfitting, a gente pegou 50 mil amostras pra treinamento e 10 mil pra validação.

In [6]:
fr_texts, pt_texts = prepare_dataset(train_examples, max_samples=50000)

In [7]:
val_fr_texts, val_pt_texts = prepare_dataset(val_examples, max_samples=10000)

## 🛠️ Configurando os Tokenizers

Criamos uma função que faz a tokenização e o padding das frases. Usamos o token `[OOV]` pra lidar com palavras que não estão no vocabulário e, depois, aplicamos padding pra deixar todas as sequências com o mesmo tamanho. Assim, fica mais fácil converter o texto em sequências numéricas e alimentar a rede neural sem complicação.

In [8]:
def tokenize_and_pad(fr_texts, pt_texts, max_input_length, max_target_length):
    tokenizer_fr = Tokenizer(filters='', oov_token='[OOV]')
    tokenizer_fr.fit_on_texts(fr_texts)
    input_vocab_size = len(tokenizer_fr.word_index) + 1

    tokenizer_pt = Tokenizer(filters='', oov_token='[OOV]')
    tokenizer_pt.fit_on_texts(pt_texts)
    target_vocab_size = len(tokenizer_pt.word_index) + 1

    # Texto para sequência
    fr_sequences = tokenizer_fr.texts_to_sequences(fr_texts)
    pt_sequences = tokenizer_pt.texts_to_sequences(pt_texts)

    # Preparar entradas inputs e decoder outputs
    decoder_inputs = [seq[:-1] for seq in pt_sequences]
    decoder_outputs = [seq[1:] for seq in pt_sequences]

    # Pad sequences
    encoder_inputs = pad_sequences(fr_sequences, maxlen=max_input_length, padding='post')
    decoder_inputs = pad_sequences(decoder_inputs, maxlen=(max_target_length - 1), padding='post')
    decoder_outputs = pad_sequences(decoder_outputs, maxlen=(max_target_length - 1), padding='post')

    return encoder_inputs, decoder_inputs, decoder_outputs, input_vocab_size, target_vocab_size


In [9]:
max_input_length = 50
max_target_length = 50

In [10]:
train_encoder_inputs, train_decoder_inputs, train_decoder_outputs, input_vocab_size, target_vocab_size = tokenize_and_pad(
    fr_texts, pt_texts, max_input_length, max_target_length
)

In [11]:
val_encoder_inputs, val_decoder_inputs, val_decoder_outputs, _, _ = tokenize_and_pad(
    val_fr_texts, val_pt_texts, max_input_length, max_target_length
)

# 🤖 Modelo 1: LSTM Básico

No modelo LSTM, definimos a função build_lstm_model que basicamente cria um encoder e um decoder usando teacher forcing. A ideia é que o encoder pegue a sequência de entrada e o decoder aprenda a gerar a saída com base nas informações que recebeu, sempre “corrigindo” o que já foi gerado. As camadas de inferência, que seriam usadas na hora de testar ou gerar traduções de verdade, a gente deixou de lado, mas o ChatGPT e o Deepseek recomendaram para gente, mas não era o foco do trabalho.

In [11]:
def build_lstm_model(input_vocab_size, target_vocab_size,
                     max_input_len, max_target_len,
                     latent_units=256, embedding_dim=256):
    # Encoder
    encoder_inputs = Input(shape=(max_input_len,), name="encoder_inputs")
    encoder_embed = Embedding(input_vocab_size, embedding_dim, name="encoder_embedding")(encoder_inputs)
    encoder_lstm = LSTM(latent_units, return_state=True, name="encoder_lstm")
    _, state_h, state_c = encoder_lstm(encoder_embed)

    # Decoder
    decoder_inputs = Input(shape=(max_target_len-1,), name="decoder_inputs")
    decoder_embed = Embedding(target_vocab_size, embedding_dim, name="decoder_embedding")(decoder_inputs)
    decoder_lstm = LSTM(latent_units, return_sequences=True, name="decoder_lstm")
    decoder_outputs = decoder_lstm(decoder_embed, initial_state=[state_h, state_c])
    decoder_dense = Dense(target_vocab_size, activation='softmax', dtype='float32', name="decoder_dense")
    decoder_outputs = decoder_dense(decoder_outputs)


    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    model.encoder_inputs = encoder_inputs
    model.encoder_embedding = model.get_layer("encoder_embedding")
    model.encoder_lstm = model.get_layer("encoder_lstm")
    model.decoder_embedding = model.get_layer("decoder_embedding")
    model.decoder_lstm = model.get_layer("decoder_lstm")
    model.decoder_dense = decoder_dense
    return model


## Criando e Compilando os modelos

Para todos os nossos modelos, a configuração foi padrão: usamos o otimizador Adam com learning rate de 1e-4 e clipnorm de 1.0, a loss foi definida como 'sparse_categorical_crossentropy' e a métrica de acurácia. Essa padronização ajudou a gente a comparar os resultados dos modelos sem ter que ficar ajustando vários parâmetros diferentes.

In [12]:
lstm_model = build_lstm_model(input_vocab_size, target_vocab_size, max_input_length, max_target_length)

lstm_model.compile(
    optimizer=Adam(learning_rate=1e-4, clipnorm=1.0),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

lstm_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs            │ (None, 50)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_inputs            │ (None, 49)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ encoder_embedding         │ (None, 50, 256)        │      9,464,576 │ encoder_inputs[0][0]   │
│ (Embedding)               │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_embedding         │ (None, 49, 256)        │      9,603,328 │ decoder_inputs[0][0]   │
│ (Embedding)               │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ encoder_lstm (LSTM)       │ [(None, 256), (None,   │        525,312 │ encoder_embedding[0][… │
│                           │ 256), (None, 256)]     │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_lstm (LSTM)       │ (None, 49, 256)        │        525,312 │ decoder_embedding[0][… │
│                           │                        │                │ encoder_lstm[0][1],    │
│                           │                        │                │ encoder_lstm[0][2]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ cast (Cast)               │ (None, 49, 256)        │              0 │ decoder_lstm[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_dense (Dense)     │ (None, 49, 37513)      │      9,640,841 │ cast[0][0]             │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 29,759,369 (113.52 MB)

 Trainable params: 29,759,369 (113.52 MB)

 Non-trainable params: 0 (0.00 B)

## Treinando o Modelo

In [27]:
# Treinamento
history = lstm_model.fit(
    [train_encoder_inputs, train_decoder_inputs], train_decoder_outputs,
    validation_data=([val_encoder_inputs, val_decoder_inputs], val_decoder_outputs),
    epochs=3,
    batch_size=10,
    verbose=1
)

Epoch 1/3
4388/4388 ━━━━━━━━━━━━━━━━━━━━ 81s 16ms/step - accuracy: 0.6579 - loss: 3.3281 - val_accuracy: 0.6961 - val_loss: 2.1554
Epoch 2/3
4388/4388 ━━━━━━━━━━━━━━━━━━━━ 75s 17ms/step - accuracy: 0.7016 - loss: 2.1311 - val_accuracy: 0.6958 - val_loss: 2.2553
Epoch 3/3
4388/4388 ━━━━━━━━━━━━━━━━━━━━ 65s 15ms/step - accuracy: 0.7076 - loss: 2.0305 - val_accuracy: 0.6932 - val_loss: 2.3375


# 🤖 Modelo 2: LSTM (Luong)

A gente adaptou o modelo LSTM para incluir um mecanismo de atenção – escolhemos o esquema de Luong mesmo, pois pareceu mais simples e direto. Aqui, em vez de usar teacher forcing, o decoder “presta atenção” nas partes relevantes do encoder durante a geração da tradução. Foi interessante notar como a atenção ajudou a melhorar o alinhamento entre as palavras de entrada e saída, mesmo com uma estrutura parecida com a LSTM simples.


In [28]:
from tensorflow.keras.layers import Layer

class ConcatenateLayer(Layer):
    def __init__(self, axis=-1, **kwargs):
        super(ConcatenateLayer, self).__init__(**kwargs)
        self.axis = axis

    def call(self, inputs):
        return tf.concat(inputs, axis=self.axis)


def build_lstm_attention_model(input_vocab_size, target_vocab_size, max_input_len, max_target_len, latent_units=256, embedding_dim=256):
    # Encoder
    encoder_inputs = Input(shape=(max_input_len,))
    encoder_embed = Embedding(input_vocab_size, embedding_dim)(encoder_inputs)
    encoder_lstm = LSTM(latent_units, return_sequences=True, return_state=True)
    encoder_outputs, state_h, state_c = encoder_lstm(encoder_embed)

    # Decoder
    decoder_inputs = Input(shape=(max_target_len - 1,))
    decoder_embed = Embedding(target_vocab_size, embedding_dim)(decoder_inputs)
    decoder_lstm = LSTM(latent_units, return_sequences=True)
    decoder_outputs = decoder_lstm(decoder_embed, initial_state=[state_h, state_c])

    # Atenção
    attention = Attention()([decoder_outputs, encoder_outputs])

    decoder_concat = ConcatenateLayer(axis=-1)([decoder_outputs, attention])
    decoder_dense = Dense(target_vocab_size, activation='softmax', dtype='float32')
    decoder_outputs = decoder_dense(decoder_concat)


    return Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [29]:
lstm_attn_model = build_lstm_attention_model(
    input_vocab_size, target_vocab_size, max_input_length, max_target_length
)

# optimizer=Adam(learning_rate=0.001)
lstm_attn_model.compile(
    optimizer=Adam(),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

lstm_attn_model.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_12            │ (None, 50)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_13            │ (None, 49)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_8 (Embedding)   │ (None, 50, 256)        │      9,464,576 │ input_layer_12[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_9 (Embedding)   │ (None, 49, 256)        │      9,603,328 │ input_layer_13[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_6 (LSTM)             │ [(None, 50, 256),      │        525,312 │ embedding_8[0][0]      │
│                           │ (None, 256), (None,    │                │                        │
│                           │ 256)]                  │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_7 (LSTM)             │ (None, 49, 256)        │        525,312 │ embedding_9[0][0],     │
│                           │                        │                │ lstm_6[0][1],          │
│                           │                        │                │ lstm_6[0][2]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention_3 (Attention)   │ (None, 49, 256)        │              0 │ lstm_7[0][0],          │
│                           │                        │                │ lstm_6[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_layer_1       │ (None, 49, 512)        │              0 │ lstm_7[0][0],          │
│ (ConcatenateLayer)        │                        │                │ attention_3[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ cast_3 (Cast)             │ (None, 49, 512)        │              0 │ concatenate_layer_1[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_10 (Dense)          │ (None, 49, 37513)      │     19,244,169 │ cast_3[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 39,362,697 (150.16 MB)

 Trainable params: 39,362,697 (150.16 MB)

 Non-trainable params: 0 (0.00 B)

In [30]:
lstm_attn_history = lstm_attn_model.fit(
    [train_encoder_inputs, train_decoder_inputs], train_decoder_outputs,
    validation_data=([val_encoder_inputs, val_decoder_inputs], val_decoder_outputs),
    epochs=3,
    batch_size=10,
    verbose=1
)

Epoch 1/3
4388/4388 ━━━━━━━━━━━━━━━━━━━━ 87s 17ms/step - accuracy: 0.6960 - loss: 2.3173 - val_accuracy: 0.6953 - val_loss: 2.4708
Epoch 2/3
4388/4388 ━━━━━━━━━━━━━━━━━━━━ 82s 19ms/step - accuracy: 0.7854 - loss: 1.3883 - val_accuracy: 0.6937 - val_loss: 2.6132
Epoch 3/3
4388/4388 ━━━━━━━━━━━━━━━━━━━━ 71s 16ms/step - accuracy: 0.8218 - loss: 1.0146 - val_accuracy: 0.6906 - val_loss: 2.7616


# 🤖 Modelo 3: Transformador

Por fim, implementamos um Transformer, que é uma abordagem totalmente diferente, sem recorrência. No código, o encoder e o decoder são feitos de várias camadas com MultiHeadAttention, normalização e dropout. Cada embedding é escalado e normalizado antes de entrar nas camadas, e no final temos uma camada densa com softmax pra prever as palavras. Esse modelo chamou nossa atenção porque, apesar de mais complexo, mostrou um potencial único pra capturar relações mais profundas nas sentenças como o professor tinha dito em sala.

Como os transformers não têm uma estrutura recorrente, não rola captar a ordem dos tokens só com o embedding. Pra resolver isso, a gente adicionou o PositionalEncoding. Criamos a função get_angles que calcula ângulos baseados na posição e na dimensão dos embeddings. Esses valores são somados aos embeddings pra dar um senso de posição pra cada token. Assim, o modelo consegue entender melhor a sequência das palavras, o que melhora a qualidade da tradução.

In [16]:

class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, max_len, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(max_len, d_model)

    def get_angles(self, pos, i, d_model):
        angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
        return pos * angle_rates

    def positional_encoding(self, max_len, d_model):
        angle_rads = self.get_angles(np.arange(max_len)[:, np.newaxis],
                                     np.arange(d_model)[np.newaxis, :],
                                     d_model)

        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
        pos_encoding = angle_rads[np.newaxis, ...]
        return tf.cast(pos_encoding, dtype=tf.float16)

    def call(self, x):
        seq_len = tf.shape(x)[1]
        return x + self.pos_encoding[:, :seq_len, :]

No transformer, a gente implementou uma máscara look-ahead pro decoder. Essa máscara cria uma matriz triangular superior, com 1s só nas posições acima da diagonal e zeros na diagonal e abaixo dela. Assim, o modelo não consegue “olhar” para as próximas palavras enquanto está gerando a tradução, o que garante que ele só use as informações que já foram produzidas. Isso é importante pra evitar vazamento de informação e manter a coerência na geração do texto.

In [19]:
def create_look_ahead_mask(size):

    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask


In [17]:
from tensorflow.keras.layers import LayerNormalization, Dropout

def build_transformer_model(input_vocab_size,
                            target_vocab_size,
                            max_input_len,
                            max_target_len,
                            num_layers=2,
                            d_model=256,
                            num_heads=8,
                            dff=512,
                            dropout_rate=0.1):

    encoder_inputs = Input(shape=(max_input_len,), name="encoder_inputs")

    enc_embed = Embedding(input_vocab_size, d_model)(encoder_inputs)
    enc_embed *= tf.math.sqrt(tf.cast(d_model, tf.float32))

    enc_embed = PositionalEncoding(max_input_len, d_model)(enc_embed)
    x = Dropout(dropout_rate)(enc_embed)

    for _ in range(num_layers):

        attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=d_model)(x, x)
        attn_output = Dropout(dropout_rate)(attn_output)
        x = LayerNormalization(epsilon=1e-6)(x + attn_output)

        ffn_output = Dense(dff, activation='relu')(x)
        ffn_output = Dense(d_model)(ffn_output)
        ffn_output = Dropout(dropout_rate)(ffn_output)
        x = LayerNormalization(epsilon=1e-6)(x + ffn_output)
    encoder_outputs = x

    decoder_inputs = Input(shape=(max_target_len,), name="decoder_inputs")
    dec_embed = Embedding(target_vocab_size, d_model)(decoder_inputs)
    dec_embed *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    dec_embed = PositionalEncoding(max_target_len, d_model)(dec_embed)
    y = Dropout(dropout_rate)(dec_embed)

    look_ahead_mask = create_look_ahead_mask(max_target_len)

    for _ in range(num_layers):

        self_attn_output = MultiHeadAttention(
            num_heads=num_heads,
            key_dim=d_model
        )(y, y, attention_mask=look_ahead_mask)
        self_attn_output = Dropout(dropout_rate)(self_attn_output)
        y = LayerNormalization(epsilon=1e-6)(y + self_attn_output)

        cross_attn_output = MultiHeadAttention(
            num_heads=num_heads,
            key_dim=d_model
        )(y, encoder_outputs)
        cross_attn_output = Dropout(dropout_rate)(cross_attn_output)
        y = LayerNormalization(epsilon=1e-6)(y + cross_attn_output)

        ffn_output = Dense(dff, activation='relu')(y)
        ffn_output = Dense(d_model)(ffn_output)
        ffn_output = Dropout(dropout_rate)(ffn_output)
        y = LayerNormalization(epsilon=1e-6)(y + ffn_output)

    decoder_outputs = y

    outputs = Dense(target_vocab_size, activation='softmax')(decoder_outputs)

    return Model([encoder_inputs, decoder_inputs], outputs)

In [14]:
print(input_vocab_size, target_vocab_size, max_input_length, max_target_length)

36971 37513 50 50


In [20]:
transformer_model = build_transformer_model(input_vocab_size, target_vocab_size, max_input_length, max_target_length - 1)

# optimizer=Adam(learning_rate=0.001)
transformer_model.compile(
    optimizer=Adam(),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

transformer_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs            │ (None, 50)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_3 (Embedding)   │ (None, 50, 256)        │      9,464,576 │ encoder_inputs[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multiply_3 (Multiply)     │ (None, 50, 256)        │              0 │ embedding_3[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ cast_3 (Cast)             │ (None, 50, 256)        │              0 │ multiply_3[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ positional_encoding_3     │ (None, 50, 256)        │              0 │ cast_3[0][0]           │
│ (PositionalEncoding)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_8 (Dropout)       │ (None, 50, 256)        │              0 │ positional_encoding_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_2    │ (None, 50, 256)        │      2,103,552 │ dropout_8[0][0],       │
│ (MultiHeadAttention)      │                        │                │ dropout_8[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_10 (Dropout)      │ (None, 50, 256)        │              0 │ multi_head_attention_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_4 (Add)               │ (None, 50, 256)        │              0 │ dropout_8[0][0],       │
│                           │                        │                │ dropout_10[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_4     │ (None, 50, 256)        │            512 │ add_4[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)           │ (None, 50, 512)        │        131,584 │ layer_normalization_4… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_5 (Dense)           │ (None, 50, 256)        │        131,328 │ dense_4[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_11 (Dropout)      │ (None, 50, 256)        │              0 │ dense_5[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_5 (Add)               │ (None, 50, 256)        │              0 │ layer_normalization_4… │
│                           │                        │                │ dropout_11[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_inputs            │ (None, 49)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_5     │ (None, 50, 256)        │            512 │ add_5[0][0]            │
│ (LayerNormalization) 

 Total params: 42,386,825 (161.69 MB)

 Trainable params: 42,386,825 (161.69 MB)

 Non-trainable params: 0 (0.00 B)

In [51]:
transformer_history = transformer_model.fit(
    [train_encoder_inputs, train_decoder_inputs], train_decoder_outputs,
    validation_data=([val_encoder_inputs, val_decoder_inputs], val_decoder_outputs),
    epochs=50,
    batch_size=10,
    verbose=1
)

Epoch 1/5
4388/4388 ━━━━━━━━━━━━━━━━━━━━ 169s 26ms/step - accuracy: 1.6326e-05 - loss: nan - val_accuracy: 0.0000e+00 - val_loss: nan
Epoch 2/5
4388/4388 ━━━━━━━━━━━━━━━━━━━━ 62s 14ms/step - accuracy: 1.0824e-05 - loss: nan - val_accuracy: 0.0000e+00 - val_loss: nan
Epoch 3/5
4388/4388 ━━━━━━━━━━━━━━━━━━━━ 62s 14ms/step - accuracy: 1.6536e-05 - loss: nan - val_accuracy: 0.0000e+00 - val_loss: nan
Epoch 4/5
4388/4388 ━━━━━━━━━━━━━━━━━━━━ 62s 14ms/step - accuracy: 1.5173e-05 - loss: nan - val_accuracy: 0.0000e+00 - val_loss: nan
Epoch 5/5
4388/4388 ━━━━━━━━━━━━━━━━━━━━ 62s 14ms/step - accuracy: 1.5460e-05 - loss: nan - val_accuracy: 0.0000e+00 - val_loss: nan


# 📊 Visualizando os Resultados

In [52]:
def visualize_results(history):
    plt.figure(figsize=(12, 4))

    # Plot da loss
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Treino')
    plt.plot(history.history['val_loss'], label='Validação')
    plt.title('Loss ao Longo do Treinamento')
    plt.xlabel('Época')
    plt.ylabel('Loss')
    plt.legend()

    # Plot da acurácia
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Treino')
    plt.plot(history.history['val_accuracy'], label='Validação')
    plt.title('Acurácia ao Longo do Treinamento')
    plt.xlabel('Época')
    plt.ylabel('Acurácia')
    plt.legend()

    plt.tight_layout()
    plt.show()

In [53]:
def plot_accuracy_by_position(y_true, y_pred):
    seq_length = y_true.shape[1]
    accuracies = []
    for pos in range(seq_length):
        pos_accuracy = np.mean(y_true[:, pos] == y_pred[:, pos])
        accuracies.append(pos_accuracy)

    plt.figure(figsize=(10, 6))
    plt.plot(range(seq_length), accuracies, marker='o')
    plt.xlabel("Token Position")
    plt.ylabel("Accuracy")
    plt.title("Accuracy by Token Position")
    plt.ylim(0, 1)
    plt.grid(True)
    plt.show()


In [54]:
def plot_length_distribution(y_true, y_pred):
    # Calculando comprimentos (ignorando padding)
    true_lengths = [len([x for x in seq if x != 0]) for seq in y_true]
    pred_lengths = [len([x for x in seq if x != 0]) for seq in y_pred]

    plt.figure(figsize=(12, 6))
    plt.hist([true_lengths, pred_lengths], label=['Real', 'Previsto'],
             alpha=0.7, bins=20)
    plt.title('Distribuição do Comprimento das Traduções')
    plt.xlabel('Comprimento da Sequência')
    plt.ylabel('Frequência')
    plt.legend()
    plt.show()

In [55]:
def plot_confusion_heatmap(y_true, y_pred, token_labels):
    """
    Plots a confusion matrix (as a heatmap) for token predictions.

    Parameters:
      - y_true: numpy array of token indices (can be 2D or flattened)
      - y_pred: numpy array of token indices (can be 2D or flattened)
      - token_labels: list of strings that maps each token index to a label.
                      For example: ["PAD", "[START]", "[END]", "bonjour", ...]
    """
    if y_true.ndim > 1:
        y_true = y_true.flatten()
    if y_pred.ndim > 1:
        y_pred = y_pred.flatten()

    # Compute the confusion matrix.
    labels = np.arange(len(token_labels))
    cm = confusion_matrix(y_true, y_pred, labels=labels)

    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                xticklabels=token_labels,
                yticklabels=token_labels)
    plt.xlabel("Predicted Token")
    plt.ylabel("True Token")
    plt.title("Confusion Matrix of Token Predictions")
    plt.show()

In [59]:
def generate_predictions(model, encoder_inputs, decoder_inputs, max_target_length, num_samples=10):
    encoder_subset = encoder_inputs[:num_samples]
    decoder_subset = decoder_inputs[:num_samples]

    predictions = model.predict([encoder_subset, decoder_subset])
    predictions = np.argmax(predictions, axis=-1)
    return predictions

# Tokens...
token_labels = list(range(1, target_vocab_size))

models = {
    "LSTM Básico": lstm_model,
    "LSTM (Luong)": lstm_attn_model,
    "Transformer": transformer_model
}

# Pegando apenas 10
num_samples = 10

In [1]:
for name, model in models.items():
    print(f"\Anlisando {name} model...")

    train_predictions = generate_predictions(model, train_encoder_inputs, train_decoder_inputs,
                                           max_target_length, num_samples)
    val_predictions = generate_predictions(model, val_encoder_inputs, val_decoder_inputs,
                                         max_target_length, num_samples)


    train_true_subset = train_decoder_outputs[:num_samples]
    val_true_subset = val_decoder_outputs[:num_samples]


    print("\Treinamento:")
    plot_accuracy_by_position(train_true_subset, train_predictions)
    plot_length_distribution(train_true_subset, train_predictions)
    #plot_confusion_heatmap(train_true_subset, train_predictions, token_labels)


    #print("\Validação:")
    #plot_accuracy_by_position(val_true_subset, val_predictions)
    #plot_length_distribution(val_true_subset, val_predictions)
    #plot_confusion_heatmap(val_true_subset, val_predictions, token_labels)

NameError: name 'models' is not defined

Depois de testar os três modelos, a gente viu que o LSTM com atenção foi o melhor em termos de acurácia e velocidade de treino. Ele chegou numa acurácia maior de forma mais rápida, e o treino foi mais ágil se comparado com o primeiro LSTM sem atenção. Mesmo que os resultados gerais sejam parecidos – e a gente talvez veja diferenças maiores na inferência – vale notar que o LSTM com atenção pesa em torno de 150 MB, enquanto o LSTM normal tem cerca de 1/3 menos, sendo assim mais eficiente em memória.

Por outro lado, o transformador se mostrou o maior de todos, com quase 190 MB de memória! Esse ponto é super relevante e precisa ser levado em consideração quando se constrói modelos parecidos. Apesar de treinar de um jeito muito rápido, o transformador demorou um pouco para atingir uma acurácia minimamente boa nas primeiras épocas. Só depois de rodar por umas 50 épocas que a taxa de crescimento se estabilizou e começaram a aparecer ganhos reais.

Obviamente a gente não tem entendimento completo sobre tudo o que foi escrito aqui, utilizamos de muitas referências, principalmente ferramentas para nos auxiliar, como o próprio Copilot do Colab para resolver vários erros que tivemos, Deepseek para dar códigos básicos para que nós construíssemos em cima algo melhor, e a própria documentação do Keras para entendermos como utilizar os paramẽtors dos Layers corretamente.

Por fim, entendemos que foi uma boa oportunidade de aprendizado para nós, não conseguimos ter alcançado sucesso total com o modelo de Transformador, pois nosso Colab simplesmente não aguentava rodar muitas épocas ser desconectado, mas entendemos como o "core" de um transformador funciona, e o quão impactante é para LLMs.

Agradecemos ao professor pela oportunidade de aprendizado.